# Notebook 05: Exploratory Data Analysis (EDA)

**Purpose**: Deep dive into monthly aggregated data to understand patterns, seasonality, and trends

**Input**: `data/processed/monthly_aggregated.csv`

**Output**: 
- Seasonal decomposition results
- Correlation analysis
- Pattern identification
- Insights for model selection

**Author**: Kevin Kuhn  
**Date**: 2025-10-17

In [9]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

from utils.traveco_utils import ConfigLoader, load_processed_data

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Analysis timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Analysis timestamp: 2025-10-17 23:02:52


## 1. Load Configuration and Data

In [10]:
# Load configuration (use relative path from notebooks directory)
config = ConfigLoader('../config/config.yaml')

# Load monthly aggregated data
df = load_processed_data('monthly_aggregated.csv', config)

# First, let's see what columns we have
print(f"\nAvailable columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

# Convert year_month to datetime (handle different possible column names)
if 'year_month' in df.columns:
    df['year_month'] = pd.to_datetime(df['year_month'])
elif 'date' in df.columns:
    df['year_month'] = pd.to_datetime(df['date'])
else:
    print(f"⚠️  Warning: No date column found. Available columns: {list(df.columns)}")

print(f"\nData shape: {df.shape}")

if 'year_month' in df.columns:
    print(f"Date range: {df['year_month'].min()} to {df['year_month'].max()}")
    print(f"Unique dates: {df['year_month'].nunique()}")

# Check for branch column (could be named differently)
branch_col = None
for possible_name in ['Niederlassung', 'branch', 'Branch', 'branch_name']:
    if possible_name in df.columns:
        branch_col = possible_name
        break

if branch_col:
    print(f"\nBranches (using '{branch_col}'): {df[branch_col].nunique()}")
else:
    print(f"\n⚠️  No branch column found - data may already be aggregated to company level")

Loaded 19 rows from: ../data/processed/monthly_aggregated.csv

Available columns: ['year_month', 'Id.Dispostelle', 'total_orders', 'total_distance_km', 'avg_distance_km', 'median_distance_km', 'external_driver_orders', 'internal_driver_orders', 'pickup_orders', 'delivery_orders', 'date', 'year', 'month', 'quarter', 'month_name', 'split']

First few rows:
  year_month               Id.Dispostelle  total_orders  total_distance_km  \
0    2025-06                            -          3544               54.0   
1    2025-06       1000_TRP_Lager Nebikon           508            41606.0   
2    2025-06              14_TRP_Oberbipp         31080          2046293.0   
3    2025-06            15_TRP_Intermodal           894            65662.0   
4    2025-06  16_TRP_Herzogenbuchsee (DP)          4312           277401.0   

   avg_distance_km  median_distance_km  external_driver_orders  \
0        18.000000                24.0                       0   
1        82.063116                83.0    

In [11]:
# Display sample data
print("\n=== Sample Data ===")
display(df.head(10))

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics ===")
display(df.describe())


=== Sample Data ===


,year_month,Id.Dispostelle,total_orders,total_distance_km,avg_distance_km,median_distance_km,external_driver_orders,internal_driver_orders,pickup_orders,delivery_orders,date,year,month,quarter,month_name,split
0,2025-06-01,-,3544,54.0,18.000000,24.0,0,2375,0,3543,2025-06-01,2025,6,2,June,train
1,2025-06-01,1000_TRP_Lager Nebikon,508,41606.0,82.063116,83.0,0,0,0,508,2025-06-01,2025,6,2,June,train
2,2025-06-01,14_TRP_Oberbipp,31080,2046293.0,65.839543,58.0,8578,22502,1254,29826,2025-06-01,2025,6,2,June,train
3,2025-06-01,15_TRP_Intermodal,894,65662.0,80.468137,54.0,290,604,374,520,2025-06-01,2025,6,2,June,train
4,2025-06-01,16_TRP_Herzogenbuchsee (DP),4312,277401.0,64.332328,55.0,262,4050,0,4312,2025-06-01,2025,6,2,June,train
5,2025-06-01,18_TRP_Puidoux (DP),888,51974.0,58.529279,51.0,0,888,0,888,2025-06-01,2025,6,2,June,train
6,2025-06-01,19_TRP_Sierre,8582,474577.0,55.299114,34.0,81,8501,2910,5672,2025-06-01,2025,6,2,June,train
7,2025-06-01,1_B&T_1100 Ohringen,5295,135804.0,25.647592,21.0,0,3722,0,5295,2025-06-01,2025,6,2,June,train
8,2025-06-01,1_TRP_Lahr,2330,9038.0,50.775281,43.0,2152,178,1488,842,2025-06-01,2025,6,2,June,train
9,2025-06-01,2_B&T_1200 Landquart,600,15897.0,26.495000,21.0,0,599,0,600,2025-06-01,2025,6,2,June,train



=== Data Types ===
year_month                datetime64[ns]
Id.Dispostelle                    object
total_orders                       int64
total_distance_km                float64
avg_distance_km                  float64
median_distance_km               float64
external_driver_orders             int64
internal_driver_orders             int64
pickup_orders                      int64
delivery_orders                    int64
date                              object
year                               int64
month                              int64
quarter                            int64
month_name                        object
split                             object
dtype: object

=== Summary Statistics ===


,year_month,total_orders,total_distance_km,avg_distance_km,median_distance_km,external_driver_orders,internal_driver_orders,pickup_orders,delivery_orders,year,month,quarter
count,19,19.000000,1.900000e+01,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.0,19.0
mean,2025-06-01 00:00:00,7166.263158,4.317172e+05,55.915842,45.894737,1304.315789,5615.684211,909.736842,6256.473684,2025.0,6.0,2.0
min,2025-06-01 00:00:00,5.000000,5.400000e+01,18.000000,1.000000,0.000000,0.000000,0.000000,5.000000,2025.0,6.0,2.0
25%,2025-06-01 00:00:00,781.000000,2.379300e+04,47.239219,35.500000,0.000000,388.500000,0.000000,637.000000,2025.0,6.0,2.0
50%,2025-06-01 00:00:00,2330.000000,1.192610e+05,55.299114,48.000000,81.000000,1612.000000,4.000000,1589.000000,2025.0,6.0,2.0
75%,2025-06-01 00:00:00,6938.500000,3.759890e+05,66.002197,58.000000,1221.000000,6275.500000,1371.000000,5483.500000,2025.0,6.0,2.0
max,2025-06-01 00:00:00,31080.000000,2.290246e+06,87.617965,83.000000,8578.000000,23373.000000,5344.000000,29826.000000,2025.0,6.0,2.0
std,NaN,10327.643201,7.126653e+05,19.560179,20.736159,2439.802680,8282.104584,1496.157963,9236.427751,0.0,0.0,0.0


## 2. Time Series Decomposition

Decompose time series into:
- **Trend**: Long-term increase/decrease
- **Seasonal**: Repeating patterns (yearly, quarterly, monthly)
- **Residual**: Random noise

In [12]:
# Aggregate to total company level for decomposition
# Check if data is already at company level or needs aggregation

# Identify branch column
branch_col = None
for possible_name in ['Niederlassung', 'branch', 'Branch', 'branch_name']:
    if possible_name in df.columns:
        branch_col = possible_name
        break

# Define target columns (check which exist)
target_cols_to_agg = {}
if 'total_orders' in df.columns:
    target_cols_to_agg['total_orders'] = 'sum'
if 'external_driver_orders' in df.columns:
    target_cols_to_agg['external_driver_orders'] = 'sum'
if 'internal_driver_orders' in df.columns:
    target_cols_to_agg['internal_driver_orders'] = 'sum'
if 'total_distance_km' in df.columns:
    target_cols_to_agg['total_distance_km'] = 'sum'

if branch_col and len(target_cols_to_agg) > 0:
    # Data has branches, aggregate to company level
    df_total = df.groupby('year_month').agg(target_cols_to_agg).reset_index()
    print(f"Aggregated from {df[branch_col].nunique()} branches to company level")
elif 'year_month' in df.columns and len(target_cols_to_agg) > 0:
    # Data appears to already be at company level
    df_total = df.copy()
    print("Data appears to already be at company level (no branch column)")
else:
    # Fallback - use all numeric columns
    print("⚠️  Using fallback aggregation")
    df_total = df.copy()

# Sort by date
if 'year_month' in df_total.columns:
    df_total = df_total.sort_values('year_month').reset_index(drop=True)
    print(f"\nTotal company time series: {len(df_total)} months")
    print(f"From {df_total['year_month'].min()} to {df_total['year_month'].max()}")
else:
    print("⚠️  Cannot sort - no year_month column")

Data appears to already be at company level (no branch column)

Total company time series: 19 months
From 2025-06-01 00:00:00 to 2025-06-01 00:00:00


In [13]:
# Seasonal decomposition for total orders
# Using additive model (better for stable variance)
# Period = 12 months for yearly seasonality

if len(df_total) >= 24:  # Need at least 2 years for good decomposition
    decomposition = seasonal_decompose(
        df_total.set_index('year_month')['total_orders'],
        model='additive',
        period=12,
        extrapolate_trend='freq'
    )
    
    # Plot decomposition
    fig, axes = plt.subplots(4, 1, figsize=(15, 10))
    
    decomposition.observed.plot(ax=axes[0], title='Observed')
    axes[0].set_ylabel('Orders')
    
    decomposition.trend.plot(ax=axes[1], title='Trend')
    axes[1].set_ylabel('Orders')
    
    decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
    axes[2].set_ylabel('Orders')
    
    decomposition.resid.plot(ax=axes[3], title='Residual')
    axes[3].set_ylabel('Orders')
    
    plt.tight_layout()
    plt.savefig('../results/seasonal_decomposition_total_orders.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate strength of trend and seasonality
    var_resid = decomposition.resid.var()
    var_trend = decomposition.trend.var()
    var_seasonal = decomposition.seasonal.var()
    
    trend_strength = max(0, 1 - var_resid / (var_resid + var_trend))
    seasonal_strength = max(0, 1 - var_resid / (var_resid + var_seasonal))
    
    print("\n=== Decomposition Analysis ===")
    print(f"Trend strength: {trend_strength:.2%}")
    print(f"Seasonal strength: {seasonal_strength:.2%}")
    print(f"\nInterpretation:")
    print(f"  - Strong trend (>70%): {'Yes' if trend_strength > 0.7 else 'No'}")
    print(f"  - Strong seasonality (>70%): {'Yes' if seasonal_strength > 0.7 else 'No'}")
else:
    print(f"⚠️  Need at least 24 months for seasonal decomposition. Current: {len(df_total)} months")

⚠️  Need at least 24 months for seasonal decomposition. Current: 19 months


## 3. Autocorrelation Analysis

Identify:
- **ACF (Autocorrelation)**: How current values relate to past values
- **PACF (Partial Autocorrelation)**: Direct relationship after removing intermediate correlations
- Useful for SARIMAX order selection (p, d, q parameters)

In [ ]:
# ACF and PACF plots
# Only run if we have enough data points

if len(df_total) >= 10:  # Need at least 10 data points for meaningful ACF/PACF
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Limit lags to at most len(data) - 2
    max_lags = min(24, len(df_total) - 2)
    
    # ACF
    plot_acf(df_total['total_orders'].dropna(), lags=max_lags, ax=axes[0])
    axes[0].set_title('Autocorrelation Function (ACF)')
    axes[0].set_xlabel('Lag (months)')
    
    # PACF
    plot_pacf(df_total['total_orders'].dropna(), lags=max_lags, ax=axes[1])
    axes[1].set_title('Partial Autocorrelation Function (PACF)')
    axes[1].set_xlabel('Lag (months)')
    
    plt.tight_layout()
    plt.savefig('../results/acf_pacf_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n=== ACF/PACF Interpretation Guide ===")
    print("For SARIMAX(p,d,q)(P,D,Q)[s] parameter selection:")
    print("  - ACF shows significant spikes at seasonal lags (12, 24) → Q parameter")
    print("  - PACF shows significant spikes at seasonal lags → P parameter")
    print("  - ACF decays gradually → MA component (q parameter)")
    print("  - PACF cuts off sharply → AR component (p parameter)")
else:
    print(f"⚠️  Skipping ACF/PACF analysis - insufficient data ({len(df_total)} time periods)")
    print(f"    Time series analysis requires multiple time periods (months/weeks)")
    print(f"    Current data appears to be from a single time period with multiple groups (branches)")

## 4. Seasonality Patterns by Month

In [ ]:
# Extract month and year for analysis
df_total['month'] = df_total['year_month'].dt.month
df_total['year'] = df_total['year_month'].dt.year

# Monthly patterns
monthly_avg = df_total.groupby('month')['total_orders'].agg(['mean', 'std', 'min', 'max']).reset_index()
monthly_avg['cv'] = monthly_avg['std'] / monthly_avg['mean']  # Coefficient of variation

# Create month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg['month_name'] = monthly_avg['month'].apply(lambda x: month_names[x-1])

print("\n=== Monthly Seasonality Patterns ===")
display(monthly_avg)

# Visualize monthly patterns
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_avg['month_name'],
    y=monthly_avg['mean'],
    error_y=dict(type='data', array=monthly_avg['std']),
    name='Average Orders',
    marker_color='lightblue'
))

fig.update_layout(
    title='Average Monthly Orders (±1 Std Dev)',
    xaxis_title='Month',
    yaxis_title='Total Orders',
    height=500,
    showlegend=False
)

fig.write_html('../results/monthly_seasonality.html')
fig.show()

# Identify peak and low months
peak_month = monthly_avg.loc[monthly_avg['mean'].idxmax(), 'month_name']
low_month = monthly_avg.loc[monthly_avg['mean'].idxmin(), 'month_name']
peak_value = monthly_avg['mean'].max()
low_value = monthly_avg['mean'].min()
seasonality_amplitude = ((peak_value - low_value) / monthly_avg['mean'].mean()) * 100

print(f"\n📊 Peak month: {peak_month} ({peak_value:.0f} orders)")
print(f"📉 Low month: {low_month} ({low_value:.0f} orders)")
print(f"📈 Seasonality amplitude: {seasonality_amplitude:.1f}%")

## 5. Year-over-Year Growth Analysis

In [ ]:
# Calculate year-over-year growth
yearly_totals = df_total.groupby('year').agg({
    'total_orders': 'sum',
    'external_driver_orders': 'sum',
    'internal_driver_orders': 'sum',
    'total_distance_km': 'sum'
}).reset_index()

yearly_totals['yoy_growth'] = yearly_totals['total_orders'].pct_change() * 100
yearly_totals['external_pct'] = (yearly_totals['external_driver_orders'] / yearly_totals['total_orders']) * 100
yearly_totals['internal_pct'] = (yearly_totals['internal_driver_orders'] / yearly_totals['total_orders']) * 100

print("\n=== Year-over-Year Analysis ===")
display(yearly_totals)

# Visualize yearly trends
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Total Orders by Year', 'YoY Growth Rate',
                   'External vs Internal Split', 'Total Distance')
)

# Total orders
fig.add_trace(
    go.Bar(x=yearly_totals['year'], y=yearly_totals['total_orders'],
           marker_color='steelblue', name='Orders'),
    row=1, col=1
)

# YoY growth
fig.add_trace(
    go.Bar(x=yearly_totals['year'][1:], y=yearly_totals['yoy_growth'][1:],
           marker_color='coral', name='Growth %'),
    row=1, col=2
)

# External vs Internal
fig.add_trace(
    go.Bar(x=yearly_totals['year'], y=yearly_totals['external_pct'],
           name='External %', marker_color='lightcoral'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=yearly_totals['year'], y=yearly_totals['internal_pct'],
           name='Internal %', marker_color='lightgreen'),
    row=2, col=1
)

# Total distance
fig.add_trace(
    go.Bar(x=yearly_totals['year'], y=yearly_totals['total_distance_km'],
           marker_color='mediumpurple', name='Distance'),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=True, title_text="Yearly Trends Analysis")
fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_xaxes(title_text="Year", row=2, col=2)
fig.update_yaxes(title_text="Orders", row=1, col=1)
fig.update_yaxes(title_text="Growth %", row=1, col=2)
fig.update_yaxes(title_text="Percentage", row=2, col=1)
fig.update_yaxes(title_text="Distance (km)", row=2, col=2)

fig.write_html('../results/yearly_trends_analysis.html')
fig.show()

# Calculate average growth rate
avg_growth = yearly_totals['yoy_growth'].mean()
print(f"\n📈 Average YoY growth rate: {avg_growth:.2f}%")

## 6. Branch-Level Correlation Analysis

In [ ]:
# Branch-Level Correlation Analysis
# Only run if we have branch-level data

# Find branch column
branch_col = None
for possible_name in ['Niederlassung', 'branch', 'Branch', 'branch_name']:
    if possible_name in df.columns:
        branch_col = possible_name
        break

if branch_col and 'total_orders' in df.columns:
    # Get top branches by total volume
    top_branches = df.groupby(branch_col)['total_orders'].sum().nlargest(10).index.tolist()
    
    # Create pivot table for correlation
    df_pivot = df[df[branch_col].isin(top_branches)].pivot(
        index='year_month',
        columns=branch_col,
        values='total_orders'
    )
    
    # Calculate correlation matrix
    correlation_matrix = df_pivot.corr()
    
    print("\n=== Branch Correlation Matrix (Top 10 Branches) ===")
    print("High correlation (>0.8) indicates similar patterns")
    print("Low correlation (<0.3) indicates independent patterns\n")
    
    # Visualize correlation matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, vmin=-1, vmax=1, square=True, linewidths=0.5)
    plt.title('Branch Order Correlation Matrix (Top 10 Branches)', fontsize=14, pad=20)
    plt.tight_layout()
    plt.savefig('../results/branch_correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Find highly correlated branch pairs
    high_corr_threshold = 0.8
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if correlation_matrix.iloc[i, j] > high_corr_threshold:
                high_corr_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if high_corr_pairs:
        print(f"\n🔗 Highly correlated branch pairs (correlation > {high_corr_threshold}):")
        for branch1, branch2, corr in sorted(high_corr_pairs, key=lambda x: x[2], reverse=True):
            print(f"  {branch1} ↔ {branch2}: {corr:.3f}")
    else:
        print(f"\nNo branch pairs with correlation > {high_corr_threshold}")
else:
    print("⚠️  Skipping branch correlation analysis - no branch column found in data")
    print("    Data appears to already be aggregated at company level")

## 7. Target Variable Correlation

In [ ]:
# Correlation between target variables
target_cols = ['total_orders', 'external_driver_orders', 'internal_driver_orders', 'total_distance_km']
available_targets = [col for col in target_cols if col in df_total.columns]

if len(available_targets) > 1:
    target_correlation = df_total[available_targets].corr()
    
    print("\n=== Target Variable Correlations ===")
    display(target_correlation)
    
    # Visualize
    plt.figure(figsize=(10, 8))
    sns.heatmap(target_correlation, annot=True, fmt='.3f', cmap='RdYlGn',
                center=0, vmin=-1, vmax=1, square=True, linewidths=1)
    plt.title('Target Variable Correlation Matrix', fontsize=14, pad=20)
    plt.tight_layout()
    plt.savefig('../results/target_correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Pairplot for visual inspection
    if len(df_total) < 500:  # Only if not too many data points
        pairplot_data = df_total[available_targets].copy()
        fig = px.scatter_matrix(
            pairplot_data,
            dimensions=available_targets,
            title='Target Variable Pairplot',
            height=800
        )
        fig.update_traces(diagonal_visible=False, showupperhalf=False)
        fig.write_html('../results/target_pairplot.html')
        fig.show()
else:
    print("Not enough target variables for correlation analysis")

## 8. Stationarity Test (Augmented Dickey-Fuller)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series, name=''):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    
    Null hypothesis: Series has a unit root (non-stationary)
    If p-value < 0.05: Reject null hypothesis (stationary)
    """
    result = adfuller(series.dropna(), autolag='AIC')
    
    print(f"\n=== ADF Test Results: {name} ===")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.3f}")
    
    if result[1] < 0.05:
        print(f"\n✅ Result: STATIONARY (p-value < 0.05)")
        print("   → Can use without differencing")
        return True
    else:
        print(f"\n❌ Result: NON-STATIONARY (p-value >= 0.05)")
        print("   → Need differencing (d=1 in SARIMAX)")
        return False

# Test original series
is_stationary = adf_test(df_total['total_orders'], 'Total Orders (Original)')

# Test first difference if non-stationary
if not is_stationary:
    df_total['total_orders_diff'] = df_total['total_orders'].diff()
    adf_test(df_total['total_orders_diff'], 'Total Orders (First Difference)')
    
    # Visualize original vs differenced
    fig, axes = plt.subplots(2, 1, figsize=(15, 8))
    
    axes[0].plot(df_total['year_month'], df_total['total_orders'])
    axes[0].set_title('Original Series')
    axes[0].set_ylabel('Total Orders')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(df_total['year_month'][1:], df_total['total_orders_diff'][1:])
    axes[1].set_title('First Difference (Δ orders)')
    axes[1].set_ylabel('Change in Orders')
    axes[1].set_xlabel('Date')
    axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/stationarity_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

## 9. Distribution Analysis

In [ ]:
# Analyze distribution of target variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

target_vars = [
    ('total_orders', 'Total Orders'),
    ('external_driver_orders', 'External Driver Orders'),
    ('internal_driver_orders', 'Internal Driver Orders'),
    ('total_distance_km', 'Total Distance (km)')
]

for idx, (col, title) in enumerate(target_vars):
    if col in df_total.columns:
        row = idx // 2
        col_idx = idx % 2
        
        data = df_total[col].dropna()
        
        # Histogram with KDE
        axes[row, col_idx].hist(data, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
        axes[row, col_idx].set_title(f'{title} Distribution')
        axes[row, col_idx].set_xlabel(title)
        axes[row, col_idx].set_ylabel('Frequency')
        
        # Add mean and median lines
        mean_val = data.mean()
        median_val = data.median()
        axes[row, col_idx].axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.0f}')
        axes[row, col_idx].axvline(median_val, color='green', linestyle='--', label=f'Median: {median_val:.0f}')
        axes[row, col_idx].legend()
        axes[row, col_idx].grid(True, alpha=0.3)
        
        # Perform normality test
        statistic, p_value = stats.shapiro(data)
        is_normal = p_value > 0.05
        normality_text = "Normal" if is_normal else "Non-normal"
        axes[row, col_idx].text(0.95, 0.95, f'{normality_text}\n(p={p_value:.3f})',
                               transform=axes[row, col_idx].transAxes,
                               verticalalignment='top', horizontalalignment='right',
                               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('../results/distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n=== Distribution Statistics ===")
for col, title in target_vars:
    if col in df_total.columns:
        data = df_total[col].dropna()
        print(f"\n{title}:")
        print(f"  Mean: {data.mean():.2f}")
        print(f"  Median: {data.median():.2f}")
        print(f"  Std Dev: {data.std():.2f}")
        print(f"  Skewness: {data.skew():.2f}")
        print(f"  Kurtosis: {data.kurtosis():.2f}")

## 10. Outlier Detection

In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(series, multiplier=1.5):
    """
    Detect outliers using Interquartile Range (IQR)
    
    multiplier=1.5: Standard outlier detection
    multiplier=3.0: Extreme outlier detection
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    
    return outliers, lower_bound, upper_bound

print("\n=== Outlier Detection (IQR Method) ===")

for col, title in target_vars:
    if col in df_total.columns:
        data = df_total[col].dropna()
        outliers, lower, upper = detect_outliers_iqr(data)
        
        print(f"\n{title}:")
        print(f"  Normal range: [{lower:.0f}, {upper:.0f}]")
        print(f"  Outliers detected: {len(outliers)}")
        
        if len(outliers) > 0:
            print(f"  Outlier values: {outliers.values}")
            outlier_dates = df_total.loc[outliers.index, 'year_month'].values
            print(f"  Outlier dates: {outlier_dates}")

# Visualize outliers
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for idx, (col, title) in enumerate(target_vars):
    if col in df_total.columns:
        row = idx // 2
        col_idx = idx % 2
        
        data = df_total[col].dropna()
        outliers, lower, upper = detect_outliers_iqr(data)
        
        # Box plot
        bp = axes[row, col_idx].boxplot([data], labels=[title], patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        
        axes[row, col_idx].set_title(f'{title} - Box Plot')
        axes[row, col_idx].set_ylabel('Value')
        axes[row, col_idx].grid(True, alpha=0.3, axis='y')
        
        # Add outlier count
        axes[row, col_idx].text(1.3, data.median(), f'{len(outliers)} outliers',
                               verticalalignment='center',
                               bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.savefig('../results/outlier_detection.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Generate EDA Summary Report

In [ ]:
# Create comprehensive EDA summary

# Find branch column
branch_col = None
for possible_name in ['Niederlassung', 'branch', 'Branch', 'branch_name']:
    if possible_name in df.columns:
        branch_col = possible_name
        break

num_branches = int(df[branch_col].nunique()) if branch_col else 1

eda_summary = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_summary': {
        'total_months': int(len(df_total)),
        'date_range': f"{df_total['year_month'].min()} to {df_total['year_month'].max()}",
        'num_branches': num_branches
    },
    'seasonality': {
        'seasonal_strength': float(seasonal_strength) if len(df_total) >= 24 else None,
        'trend_strength': float(trend_strength) if len(df_total) >= 24 else None,
        'peak_month': peak_month,
        'low_month': low_month,
        'seasonality_amplitude_pct': float(seasonality_amplitude)
    },
    'growth': {
        'avg_yoy_growth_pct': float(avg_growth) if not np.isnan(avg_growth) else None,
        'yearly_data': yearly_totals.to_dict('records')
    },
    'stationarity': {
        'is_stationary': bool(is_stationary),
        'recommended_differencing': 0 if is_stationary else 1
    },
    'model_recommendations': {}
}

# Add model recommendations based on findings
recommendations = []

if len(df_total) >= 24:
    if seasonal_strength > 0.7:
        recommendations.append("Strong seasonality detected → Prophet and SARIMAX recommended")
        recommendations.append("Use seasonal order (P,D,Q)[12] in SARIMAX")
    else:
        recommendations.append("Weak seasonality → Consider simpler models first")
else:
    recommendations.append(f"Limited data ({len(df_total)} months) → Use simpler models, avoid complex seasonal models")

if not is_stationary:
    recommendations.append("Non-stationary series → Use d=1 in SARIMAX")
else:
    recommendations.append("Stationary series → Can use d=0 in SARIMAX")

if not np.isnan(avg_growth) and abs(avg_growth) > 5:
    recommendations.append(f"Strong growth trend ({avg_growth:.1f}%) → Consider linear trend component")
else:
    recommendations.append("Stable growth → Standard trend handling sufficient")

eda_summary['model_recommendations']['insights'] = recommendations

# Save summary to JSON
with open('../results/eda_summary.json', 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    def convert_types(obj):
        if isinstance(obj, dict):
            return {k: convert_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_types(item) for item in obj]
        elif isinstance(obj, (np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif pd.isna(obj):
            return None
        else:
            return obj
    
    eda_summary_clean = convert_types(eda_summary)
    json.dump(eda_summary_clean, f, indent=2)

print("\n" + "="*70)
print("📊 EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*70)

print(f"\n📅 Data Coverage:")
print(f"   {eda_summary['data_summary']['total_months']} months")
print(f"   {eda_summary['data_summary']['date_range']}")
print(f"   {eda_summary['data_summary']['num_branches']} branches")

if eda_summary['seasonality']['seasonal_strength']:
    print(f"\n📈 Seasonality:")
    print(f"   Strength: {eda_summary['seasonality']['seasonal_strength']:.1%}")
    print(f"   Peak month: {eda_summary['seasonality']['peak_month']}")
    print(f"   Low month: {eda_summary['seasonality']['low_month']}")
    print(f"   Amplitude: {eda_summary['seasonality']['seasonality_amplitude_pct']:.1f}%")

if eda_summary['growth']['avg_yoy_growth_pct']:
    print(f"\n📊 Growth:")
    print(f"   Average YoY: {eda_summary['growth']['avg_yoy_growth_pct']:.2f}%")

print(f"\n🔍 Stationarity:")
print(f"   {'Stationary' if eda_summary['stationarity']['is_stationary'] else 'Non-stationary'}")
print(f"   Recommended differencing: d={eda_summary['stationarity']['recommended_differencing']}")

print(f"\n💡 Model Recommendations:")
for rec in recommendations:
    print(f"   • {rec}")

print("\n" + "="*70)
print("✅ EDA Complete!")
print("="*70)
print("\n📁 Saved:")
print("   - results/eda_summary.json")
if len(df_total) >= 24:
    print("   - results/seasonal_decomposition_total_orders.png")
print("   - results/acf_pacf_analysis.png")
print("   - results/monthly_seasonality.html")
print("   - results/yearly_trends_analysis.html")
if branch_col:
    print("   - results/branch_correlation_matrix.png")
print("   - results/target_correlation_matrix.png")
if len(df_total) < 500:
    print("   - results/target_pairplot.html")
print("   - results/distribution_analysis.png")
print("   - results/outlier_detection.png")
print("\n➡️  NEXT STEPS:")
print("   Proceed to notebook 06: Baseline Models (Prophet)")